In [1]:
# pbp_parse.py
#
# parse JSON, make game structure, convert to CSV.
# JSON 데이터를 읽어와 게임 상황을 구현하고
# gameday line 하나에 맞는 text row를 생성, CSV 파일에 저장한다.

import os
import json
from collections import OrderedDict
import regex
import csv

# custom library
from utils import print_progress

%load_ext autoreload
%autoreload 2

In [24]:
import pandas as pd
import numpy as np
game_id = '20190331SKWO02019'
pl = '_pitching.csv'
bl = '_batting.csv'
rl = '_relay.csv'

pdf = pd.read_csv(f'pbp_data/2019/3/{game_id}{pl}', encoding='cp949')
bdf = pd.read_csv(f'pbp_data/2019/3/{game_id}{bl}', encoding='cp949')
rdf = pd.read_csv(f'pbp_data/2019/3/{game_id}{rl}', encoding='cp949')

In [25]:
game_date = game_id[:8]
away = game_id[8:10]
home = game_id[10:12]

In [26]:
tcol = ['textOrder', 'seqno', 'text', 'type', 'stuff', 'pitchNum',
        'pitchResult', 'pitchId', 'speed', 'referee', 'stadium', 'inn',
        'ballcount', 'stance']
pcol = ['pitchId', 'speed', 'crossPlateX',
        'vx0', 'vy0', 'vz0', 'x0', 'z0', 'ax', 'ay', 'az',
        'topSz', 'bottomSz']

In [177]:
away_order = []
home_order = []
away_field = {}
home_field = {}

abat = bdf.loc[bdf.homeaway == 'a']
hbat = bdf.loc[bdf.homeaway == 'h']

apit = pdf.loc[pdf.homeaway == 'a']
hpit = pdf.loc[pdf.homeaway == 'h']

abat_seqno_min = abat.groupby('batOrder').min().seqno.tolist()
hbat_seqno_min = hbat.groupby('batOrder').min().seqno.tolist()

for i in range(9):
    aCode = abat.loc[(abat.batOrder == i+1) & (abat.seqno == abat_seqno_min[i])].pCode.values[0]
    hCode = hbat.loc[(hbat.batOrder == i+1) & (hbat.seqno == hbat_seqno_min[i])].pCode.values[0]
    away_order.append(aCode)
    home_order.append(hCode)
    
    aPos = abat.loc[abat.pCode == aCode].posName.values[0]
    hPos = hbat.loc[hbat.pCode == hCode].posName.values[0]
    away_field[aPos] = aCode
    home_field[hPos] = hCode

away_field['투수'] = apit.iloc[0].pCode
home_field['투수'] = hpit.iloc[0].pCode

In [183]:
home_score, away_score = 0, 0
ball, strike, out = 0, 0, 0
base1, base2, base3 = None, None, None


`pitchResult`
- `T` : s __T__ rike 스트라이크
- `F` : __F__oul 파울
- `S` : __S__wing 헛스윙
- `B` : __B__all 볼
- `H` : __H__it 타격

`type`
- `0` : x회초(말) xx 공격
- `1` : 투구 (x구 ...)
- `2` : 교체
- `7` : 시스템 메시지 (비디오 판독, ...)
- `8` : x번타자 xxx
- `13` : 타자주자 타석 결과(득점 없는 타석)
- `14` : 주자 주루/아웃(득점 없는 타석)
- `23` : 타자주자 타석 결과(득점 있는 타석)
- `24` : 주자 주루/아웃(득점 있는 타석)
- `44` : 파울 에러 (pass)
- `99` : 경기 종료 메시지

- `pitchNum` : 문자중계 텍스트(`textOptionList`)에 포함되는 투구수, 정합성 확인 안됨, 신뢰하지 말 것
- `ballcount` : pitch data(`ptsOptionList`)에 포함되는 투구수, `ptsPitchId` 오류 있을 때는 1개씩 밀리는 문제, 신뢰하지 말 것
- `inn` : 문자중계와 pitch data에 모두 포함, 지금은 Pitch data에 있는걸 가져오는 중. 신뢰하지 말 것

In [ ]:


for i in range(relay_len):
    if 

In [181]:
rdf[tcol].head(20)

,textOrder,seqno,text,type,stuff,pitchNum,pitchResult,pitchId,speed,referee,stadium,inn,ballcount,stance
0,0,0,1회초 SK 공격,0,NaN,NaN,NaN,NaN,NaN,우효동,고척,NaN,NaN,NaN
1,1,1,1번타자 노수광,8,NaN,NaN,NaN,NaN,NaN,우효동,고척,NaN,NaN,NaN
2,1,2,1구 스트라이크,1,슬라이더,1.0,T,190331_140049,137.0,우효동,고척,1.0,1.0,L
3,1,3,2구 스트라이크,1,투심,2.0,T,190331_140108,142.0,우효동,고척,1.0,2.0,L
4,1,4,3구 파울,1,체인지업,3.0,F,190331_140127,125.0,우효동,고척,1.0,3.0,L
5,1,5,4구 헛스윙,1,체인지업,4.0,S,190331_140203,125.0,우효동,고척,1.0,4.0,L
6,1,6,노수광 : 삼진 아웃,13,NaN,NaN,NaN,NaN,NaN,우효동,고척,NaN,NaN,NaN
7,2,10,3구 파울,1,슬라이더,3.0,F,190331_140321,137.0,우효동,고척,1.0,3.0,L
8,2,11,4구 파울,1,투심,4.0,F,190331_140350,143.0,우효동,고척,1.0,4.0,L
9,2,12,5구 헛스윙,1,체인지업,5.0,S,190331_140423,124.0,우효동,고척,1.0,5.0,L
